In [18]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [19]:
# Basics
import tensorflow as tf
import pandas as pd
import xarray as xr

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.DRN.DRN_load_score import *  # Load DRN_scores
from src.models.DRN.DRN_load_model import *  # Load Models
from src.models.DRN.DRN_train import *
from src.models.EMOS_local.EMOS_local_load_score import *  # Load EMOS_local_scores
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models#
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores

<IPython.core.display.Javascript object>

### 1. Load train-test_data

In [20]:
lead_time = 0
var_num = 5

<IPython.core.display.Javascript object>

In [21]:
# Load all training data of each variable
train_var_denormed, val_var_denormed = ldpd.load_data_all_train_val_proc_denorm()

# Split the loaded data into features (X) and target (y)
# also adjusts for lead_time
dat_X_train_lead_all_denorm, dat_y_train_lead_all_denorm = split_var_lead(
    train_var_denormed
)

# Split the loaded data into features (X) and target (y)
# also adjusts for lead_time
dat_X_val_lead_all_denorm, dat_y_val_lead_all_denorm = split_var_lead(val_var_denormed)

# Preprocess the features for Neural Network and scale them
drn_X_train_lead_array, drn_embedding_train_lead_array = make_X_array(
    dat_X_train_lead_all_denorm, lead_time
)

# Preprocess the features for Neural Network and scale them
drn_X_val_lead_array, drn_embedding_val_lead_array = make_X_array(
    dat_X_val_lead_all_denorm, lead_time
)

# Reshape target values into a 1D array
t2m_y_train = dat_y_train_lead_all_denorm[var_num][lead_time].values.flatten()


# Reshape target values into a 1D array
t2m_y_val = dat_y_val_lead_all_denorm[var_num][lead_time].values.flatten()

# load test data
test_var_denormed = ldpd.load_data_all_test_proc_denorm()
dat_X_test_lead_all_denorm, dat_y_test_lead_all_denorm = split_var_lead(
    test_var_denormed
)
drn_X_test_lead_array, drn_embedding_test_lead_array = make_X_array(
    dat_X_test_lead_all_denorm, 0
)
# Reshape target values into a 1D array
t2m_y_test = dat_y_test_lead_all_denorm[5][0].values.flatten()

<IPython.core.display.Javascript object>

### 2. Try train model:

In [22]:
DRN_0_model_test = DRN_train(
    5,
    0,
    hidden_layer=[1024],
    emb_size=10,
    batch_size=4096,
    epochs=5,
    lr=0.001,
    save=False,
    validation_split=0,
)

Epoch 1/5
4083/4083 [==============================] - 43s 10ms/step - loss: 0.7029
Epoch 2/5
4083/4083 [==============================] - 42s 10ms/step - loss: 0.4976
Epoch 3/5
4083/4083 [==============================] - 42s 10ms/step - loss: 0.4289
Epoch 4/5
4083/4083 [==============================] - 42s 10ms/step - loss: 0.4172
Epoch 5/5
4083/4083 [==============================] - 42s 10ms/step - loss: 0.4126


<IPython.core.display.Javascript object>

In [11]:
# Evaluate
DRN_0_test_score = DRN_0_model_test.evaluate(
    [drn_X_val_lead_array, drn_embedding_val_lead_array], t2m_y_val
)

174038/174038 [==============================] - 220s 1ms/step - loss: 0.4251


<IPython.core.display.Javascript object>

In [13]:
# Evaluate
DRN_0_test_score = DRN_0_model_test.evaluate(
    [drn_X_test_lead_array, drn_embedding_test_lead_array], t2m_y_test
)

174038/174038 [==============================] - 220s 1ms/step - loss: 0.6160


<IPython.core.display.Javascript object>